# Capstone Project - The Battle of the Neighborhoods (Week 2)¶

## Introduction: Business Problem

Which city is recommended for investors to open a restaurant? Toronto or New York?

## Data

Toronto:

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
This contains the Postal Code, Borough and Neighborhood information in Toronto. This can be copied and pasted into an excel file. After that, it can be loaded into notebook.

http://cocl.us/Geospatial_data
This contains the latitude and longitude of the Postal Code in Toronto.

New York:

https://geo.nyu.edu/catalog/nyu_2451_34572
This contains the Borough, Neighborhood and geographic location information in New York.

## Analysis

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 1. Get New York Data

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Use geopy library to get the latitude and longitude values of New York City.

In [9]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### 2. Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = 'T4Y134NVVAIMJB4UNHS10IMCIFY3TRJKKM4FRAATCFCAN5OZ' # your Foursquare ID
CLIENT_SECRET = '1SDBLKWT5UYT5PMTAWLLFGWUNFHLNDKE3WYA5XYG34JB01PP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T4Y134NVVAIMJB4UNHS10IMCIFY3TRJKKM4FRAATCFCAN5OZ
CLIENT_SECRET:1SDBLKWT5UYT5PMTAWLLFGWUNFHLNDKE3WYA5XYG34JB01PP


In [14]:
neighborhood_latitude_ny = neighborhoods['Latitude'] # neighborhood latitude value
neighborhood_longitude_ny = neighborhoods['Longitude'] # neighborhood longitude value

neighborhood_name_ny = neighborhoods['Neighborhood'] # neighborhood name

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [18]:
ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [19]:
print(ny_venues.shape)
ny_venues.head()

(10074, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [21]:
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,28,28,28,28,28,28
Annadale,6,6,6,6,6,6
Arden Heights,6,6,6,6,6,6
Arlington,6,6,6,6,6,6
Arrochar,22,22,22,22,22,22
Arverne,21,21,21,21,21,21
Astoria,97,97,97,97,97,97
Astoria Heights,12,12,12,12,12,12
Auburndale,20,20,20,20,20,20


In [22]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 427 uniques categories.


### 3. Analyze Each New York Neighborhoods

In [25]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
manhattan_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other

In [27]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()

Let's put that into a pandas dataframe. First, let's write a function to sort the venues in descending order.

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Spa,Supermarket,Department Store,Playground,Grocery Store,Check Cashing Service,Bus Station,Fast Food Restaurant
1,Annadale,Pizza Place,Pharmacy,Diner,Restaurant,Train Station,Food,Yoga Studio,Eye Doctor,Factory,Falafel Restaurant
2,Arden Heights,Dry Cleaner,Deli / Bodega,Pharmacy,Coffee Shop,Bus Stop,Pizza Place,Fish Market,Falafel Restaurant,Farm,Farmers Market
3,Arlington,Bus Stop,Deli / Bodega,Coffee Shop,American Restaurant,Intersection,Flea Market,Factory,Falafel Restaurant,Farm,Farmers Market
4,Arrochar,Pizza Place,Deli / Bodega,Liquor Store,Bus Stop,Italian Restaurant,Supermarket,Cosmetics Shop,Outdoors & Recreation,Sandwich Place,Mediterranean Restaurant


### 4. Cluster New York Neighborhoods

In [33]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 0, 1, 3, 3, 3, 3, 1], dtype=int32)

In [36]:
# add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head() # check the last columns

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,1.0,Pharmacy,Ice Cream Shop,Gas Station,Food,Sandwich Place,Laundromat,Donut Shop,Deli / Bodega,Dessert Shop,Distillery
1,Bronx,Co-op City,40.874294,-73.829939,1.0,Baseball Field,Pizza Place,Grocery Store,Fast Food Restaurant,Bagel Shop,Pharmacy,Restaurant,Post Office,Discount Store,Bus Station
2,Bronx,Eastchester,40.887556,-73.827806,1.0,Bus Station,Caribbean Restaurant,Deli / Bodega,Diner,Pizza Place,Donut Shop,Bowling Alley,Convenience Store,Food & Drink Shop,Chinese Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,3.0,Plaza,Music Venue,Bus Station,River,Fish Market,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market
4,Bronx,Riverdale,40.890834,-73.912585,3.0,Bus Station,Park,Bank,Food Truck,Home Service,Plaza,Medical Supply Store,Gym,Yoga Studio,Filipino Restaurant


In [39]:
ny_merged = ny_merged[ny_merged['Cluster Labels'].notnull()]

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster1

In [94]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,East New York,Deli / Bodega,Fried Chicken Joint,Plaza,Event Service,Metro Station,Fast Food Restaurant,Caribbean Restaurant,Salon / Barbershop,Pizza Place,Bus Stop
150,Whitestone,Dance Studio,Deli / Bodega,Candy Store,Bubble Tea Shop,Island,Italian Restaurant,Factory,Falafel Restaurant,Farm,Farmers Market
172,Breezy Point,Beach,Monument / Landmark,Supermarket,Bus Stop,Trail,Flea Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
179,Neponsit,Beach,Bar,Lounge,Yoga Studio,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
193,Brookville,Deli / Bodega,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service
198,New Brighton,Bus Stop,Deli / Bodega,Park,Discount Store,Flower Shop,Bowling Alley,Playground,French Restaurant,Fish Market,Farm
202,Grymes Hill,Dog Run,Deli / Bodega,Bus Stop,Flea Market,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
204,South Beach,Beach,Pier,Deli / Bodega,Bus Stop,Athletics & Sports,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
206,Mariner's Harbor,Italian Restaurant,Deli / Bodega,Supermarket,Bus Stop,Athletics & Sports,Israeli Restaurant,Flea Market,Factory,Falafel Restaurant,Farm
217,Tottenville,Italian Restaurant,Bus Stop,Mexican Restaurant,Home Service,Cosmetics Shop,Thrift / Vintage Store,Deli / Bodega,French Restaurant,Financial or Legal Service,Factory


Cluster2

In [95]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Pharmacy,Ice Cream Shop,Gas Station,Food,Sandwich Place,Laundromat,Donut Shop,Deli / Bodega,Dessert Shop,Distillery
1,Co-op City,Baseball Field,Pizza Place,Grocery Store,Fast Food Restaurant,Bagel Shop,Pharmacy,Restaurant,Post Office,Discount Store,Bus Station
2,Eastchester,Bus Station,Caribbean Restaurant,Deli / Bodega,Diner,Pizza Place,Donut Shop,Bowling Alley,Convenience Store,Food & Drink Shop,Chinese Restaurant
5,Kingsbridge,Pizza Place,Bar,Latin American Restaurant,Mexican Restaurant,Fried Chicken Joint,Bakery,Supermarket,Sandwich Place,Donut Shop,Spanish Restaurant
7,Woodlawn,Deli / Bodega,Pizza Place,Playground,Pub,Bus Stop,Food & Drink Shop,Grocery Store,Park,Bar,Pharmacy
8,Norwood,Pizza Place,Bus Station,Park,Bank,Chinese Restaurant,Pharmacy,Convenience Store,Coffee Shop,Mexican Restaurant,Sandwich Place
10,Baychester,Donut Shop,Shopping Mall,Pet Store,Bank,Other Great Outdoors,Electronics Store,Fast Food Restaurant,Pizza Place,Convenience Store,Supermarket
11,Pelham Parkway,Italian Restaurant,Pizza Place,Deli / Bodega,Frozen Yogurt Shop,Bank,Food,Gas Station,Sushi Restaurant,Smoke Shop,Chinese Restaurant
13,Bedford Park,Chinese Restaurant,Diner,Pizza Place,Mexican Restaurant,Supermarket,Deli / Bodega,Sandwich Place,Food Truck,Fast Food Restaurant,Gym
14,University Heights,Pizza Place,Fried Chicken Joint,Bakery,Supermarket,Fast Food Restaurant,Grocery Store,Shoe Store,Clothing Store,Pharmacy,Bank


Cluster3

In [96]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
192,Somerville,Park,Yoga Studio,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant
203,Todt Hill,Park,Yoga Studio,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant


Cluster4

In [97]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Plaza,Music Venue,Bus Station,River,Fish Market,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market
4,Riverdale,Bus Station,Park,Bank,Food Truck,Home Service,Plaza,Medical Supply Store,Gym,Yoga Studio,Filipino Restaurant
6,Marble Hill,Coffee Shop,Gym,Sandwich Place,Big Box Store,Miscellaneous Shop,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant,Yoga Studio
9,Williamsbridge,Nightclub,Bar,Caribbean Restaurant,Soup Place,Fish Market,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
12,City Island,Seafood Restaurant,Grocery Store,Thrift / Vintage Store,Pizza Place,Park,Harbor / Marina,Boat or Ferry,Baseball Field,Bar,Bank
18,West Farms,Bus Station,Bus Stop,Park,Donut Shop,Hotel,Outdoors & Recreation,Metro Station,Coffee Shop,Scenic Lookout,Sandwich Place
22,Port Morris,Furniture / Home Store,Peruvian Restaurant,Distillery,Grocery Store,Latin American Restaurant,Donut Shop,Baseball Field,Restaurant,Brewery,Food Truck
27,Clason Point,Park,Boat or Ferry,Bus Stop,Pool,Convenience Store,Grocery Store,South American Restaurant,Fish Market,Falafel Restaurant,Farm
28,Throgs Neck,Italian Restaurant,Asian Restaurant,Pizza Place,Coffee Shop,Sports Bar,Bar,Deli / Bodega,American Restaurant,Insurance Office,Fish & Chips Shop
29,Country Club,Playground,Sandwich Place,Athletics & Sports,Event Space,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


Cluster5

In [98]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
303,Bayswater,Construction & Landscaping,Playground,Yoga Studio,Fish Market,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


### 5. Get Toronto Data

In [48]:
df = pd.read_excel("Toronto.xlsx")
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [49]:
df = df[df['Borough'] != 'Not assigned']

In [50]:
for i in range(103):
    if df.Neighbourhood.iloc[i] == 'Not assigned':
        df.Neighbourhood.iloc[i] = df.Borough.iloc[i]

In [51]:
(df.Neighbourhood == 'Not assigned').mean() # there are no neighbourhood cells with 'not assigned'

0.0

In [52]:
df.shape

(103, 3)

In [53]:
df_geo = pd.read_csv("Geospatial_Coordinates.csv")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [54]:
latitude_list = []
for i in range(103):
    latitude_list.append(0)

In [55]:
longitude_list = []
for i in range(103):
    longitude_list.append(0)

In [56]:
df['Latitude'] = latitude_list
df['Longitude'] = longitude_list
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,0,0
3,M4A,North York,Victoria Village,0,0
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0
5,M6A,North York,"Lawrence Manor, Lawrence Heights",0,0
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0,0


In [57]:
# assign latitude and longitude to the postal codes
for i in range(103):
    for j in range (103):
        if df['Postal Code'].iloc[i] == df_geo['Postal Code'].iloc[j]:
            df['Latitude'].iloc[i] = df_geo['Latitude'].iloc[j]
            df['Longitude'].iloc[i] = df_geo['Longitude'].iloc[j]

/Users/xiaotongliang/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [58]:
df.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

In [59]:
neighborhood_latitude_tr = df['Latitude'] # neighborhood latitude value
neighborhood_longitude_tr = df['Longitude'] # neighborhood longitude value

neighborhood_name_tr = df['Neighborhood'] # neighborhood name

In [67]:
tr_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [68]:
# Let's check how many venues were returned for each neighborhood
tr_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,58,58,58,58,58,58
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19


In [69]:
print('There are {} uniques categories.'.format(len(tr_venues['Venue Category'].unique())))

There are 269 uniques categories.


### 6. Analyze Each Toronto Neighborhoods

In [71]:
# one hot encoding
tr_onehot = pd.get_dummies(tr_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tr_onehot['Neighborhood'] = tr_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tr_onehot.columns[-1]] + list(tr_onehot.columns[:-1])
tr_onehot = tr_onehot[fixed_columns]

tr_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [72]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
tr_grouped = tr_onehot.groupby('Neighborhood').mean().reset_index()
tr_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [75]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_tr = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_tr['Neighborhood'] = tr_grouped['Neighborhood']

for ind in np.arange(tr_grouped.shape[0]):
    neighborhoods_venues_sorted_tr.iloc[ind, 1:] = return_most_common_venues(tr_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_tr.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Lounge,Clothing Store,Latin American Restaurant,Breakfast Spot,Women's Store,Diner,Deli / Bodega,Department Store,Dessert Shop
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Gym,Pub,Pharmacy,Curling Ice,Dance Studio,Deli / Bodega
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Diner,Sandwich Place,Deli / Bodega,Bridal Shop,Restaurant,Ice Cream Shop,Supermarket
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store,Diner,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Grocery Store,Pizza Place,Locksmith,Juice Bar,Fast Food Restaurant,Indian Restaurant


### 7. Cluster Toronto Neighborhoods

In [76]:
tr_grouped_clustering = tr_grouped.drop('Neighborhood', 1)

# run k-means clustering
tr_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tr_grouped_clustering)

# check cluster labels generated for each row in the dataframe
tr_kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [78]:
neighborhoods_venues_sorted_tr.insert(0, 'Cluster Labels', tr_kmeans.labels_)

tr_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tr_merged = tr_merged.join(neighborhoods_venues_sorted_tr.set_index('Neighborhood'), on='Neighborhood')

tr_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Dim Sum Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Eastern European Restaurant
3,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,French Restaurant,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Creperie
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Bakery,Park,Pub,Theater,Breakfast Spot,Café,Hotel,Ice Cream Shop,Spa
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Furniture / Home Store,Clothing Store,Coffee Shop,Event Space,Boutique,Miscellaneous Shop,Vietnamese Restaurant,Accessories Store,Doner Restaurant,Donut Shop
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,College Cafeteria,Diner,Yoga Studio,Arts & Crafts Store,Distribution Center,Smoothie Shop,Beer Bar,Burrito Place,Italian Restaurant


In [79]:
tr_merged = tr_merged[tr_merged['Cluster Labels'].notnull()]

In [81]:
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tr_merged['Latitude'], tr_merged['Longitude'], tr_merged['Neighborhood'], tr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster1

In [89]:
tr_merged.loc[tr_merged['Cluster Labels'] == 0, tr_merged.columns[[1] + list(range(5, tr_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,0.0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,French Restaurant,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Creperie
12,East York,0.0,Pizza Place,Intersection,Bank,Gym / Fitness Center,Pet Store,Pharmacy,Gastropub,Athletics & Sports,Fast Food Restaurant,Dessert Shop
14,North York,0.0,Asian Restaurant,Pizza Place,Pub,Japanese Restaurant,Women's Store,Dessert Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store
80,North York,0.0,Gym,Pizza Place,Women's Store,Diner,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
102,Central Toronto,0.0,Park,Sandwich Place,Food & Drink Shop,Hotel,Gym / Fitness Center,Dance Studio,Pizza Place,Breakfast Spot,Department Store,Deli / Bodega
107,Etobicoke,0.0,Pizza Place,Coffee Shop,Discount Store,Intersection,Sandwich Place,Chinese Restaurant,Middle Eastern Restaurant,Distribution Center,Dog Run,Curling Ice
109,North York,0.0,Grocery Store,Pharmacy,Bank,Pizza Place,Butcher,Coffee Shop,College Rec Center,College Gym,Donut Shop,Doner Restaurant
116,Etobicoke,0.0,Bus Line,Pizza Place,Sandwich Place,Mobile Phone Shop,Department Store,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Women's Store
126,Scarborough,0.0,Pizza Place,Bank,Noodle House,Italian Restaurant,Pharmacy,Gas Station,Chinese Restaurant,Fried Chicken Joint,Thai Restaurant,Fast Food Restaurant
143,Etobicoke,0.0,Grocery Store,Pizza Place,Fast Food Restaurant,Beer Store,Pharmacy,Fried Chicken Joint,Sandwich Place,Dance Studio,Deli / Bodega,Department Store


Cluster2

In [90]:
tr_merged.loc[tr_merged['Cluster Labels'] == 1, tr_merged.columns[[1] + list(range(5, tr_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,1.0,Coffee Shop,Bakery,Park,Pub,Theater,Breakfast Spot,Café,Hotel,Ice Cream Shop,Spa
5,North York,1.0,Furniture / Home Store,Clothing Store,Coffee Shop,Event Space,Boutique,Miscellaneous Shop,Vietnamese Restaurant,Accessories Store,Doner Restaurant,Donut Shop
6,Downtown Toronto,1.0,Coffee Shop,College Cafeteria,Diner,Yoga Studio,Arts & Crafts Store,Distribution Center,Smoothie Shop,Beer Bar,Burrito Place,Italian Restaurant
11,North York,1.0,Gym,Coffee Shop,Restaurant,Beer Store,Japanese Restaurant,Clothing Store,Supermarket,Chinese Restaurant,Caribbean Restaurant,Café
13,Downtown Toronto,1.0,Clothing Store,Coffee Shop,Hotel,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Café,Bubble Tea Shop,Bookstore,Lingerie Store
18,Scarborough,1.0,Bar,Home Service,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Curling Ice
20,North York,1.0,Gym,Coffee Shop,Restaurant,Beer Store,Japanese Restaurant,Clothing Store,Supermarket,Chinese Restaurant,Caribbean Restaurant,Café
21,East York,1.0,Skating Rink,Pharmacy,Curling Ice,Athletics & Sports,Bus Stop,Beer Store,Park,Donut Shop,Doner Restaurant,Dog Run
22,Downtown Toronto,1.0,Café,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Restaurant,Cocktail Bar,Breakfast Spot,Beer Bar,Moroccan Restaurant
23,York,1.0,Field,Playground,Hockey Arena,Trail,Dim Sum Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop


Cluster3

In [91]:
tr_merged.loc[tr_merged['Cluster Labels'] == 2, tr_merged.columns[[1] + list(range(5, tr_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,2.0,Fast Food Restaurant,Women's Store,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center


Cluster4

In [92]:
tr_merged.loc[tr_merged['Cluster Labels'] == 3, tr_merged.columns[[1] + list(range(5, tr_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
89,North York,3.0,Baseball Field,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store,Ethiopian Restaurant
169,Etobicoke,3.0,Baseball Field,Business Service,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Curling Ice,Discount Store


Cluster5

In [93]:
tr_merged.loc[tr_merged['Cluster Labels'] == 4, tr_merged.columns[[1] + list(range(5, tr_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,North York,4.0,Park,Food & Drink Shop,Dim Sum Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Eastern European Restaurant
32,York,4.0,Women's Store,Convenience Store,Pool,Park,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
57,East York,4.0,Park,Convenience Store,Dim Sum Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Cuban Restaurant
77,North York,4.0,Park,Construction & Landscaping,Bakery,Basketball Court,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
93,Central Toronto,4.0,Park,Construction & Landscaping,Bus Line,Swim School,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Cupcake Shop
98,York,4.0,Park,Convenience Store,Dim Sum Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Cuban Restaurant
100,North York,4.0,Park,Convenience Store,Construction & Landscaping,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Cupcake Shop
103,Central Toronto,4.0,Park,Jewelry Store,Trail,Sushi Restaurant,Doner Restaurant,Dog Run,Distribution Center,Donut Shop,Drugstore,Cupcake Shop
129,Central Toronto,4.0,Park,Trail,Playground,Lawyer,German Restaurant,Drugstore,Doner Restaurant,Dog Run,Distribution Center,Discount Store
135,Scarborough,4.0,Park,Playground,Dim Sum Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Women's Store
